In [2]:
import itk
import os
import matplotlib
import matplotlib.pyplot as plt

## Lecture des données

In [8]:
file1_path = "Data/case6_gre1.nrrd"
file2_path = "Data/case6_gre2.nrrd"

PixelType = itk.F
fixed_image = itk.imread(file1_path, PixelType)
moving_image = itk.imread(file2_path, PixelType)
itk.imwrite(fixed_image, "fixed_image.nii.gz")
itk.imwrite(moving_image, "moving_image.nii.gz")

## Recalage d'images

### Recalage par translation

In [79]:
dimension = fixed_image.GetImageDimension()
FixedImageType = type(fixed_image)
MovingImageType = type(moving_image)

#Définir la transformation
TransformType = itk.TranslationTransform[itk.D, dimension]
initial_transform = TransformType.New()
initial_transform.SetIdentity()

#Définir l'optimiseur
optimizer = itk.RegularStepGradientDescentOptimizerv4.New()
optimizer.SetLearningRate(4.0)
optimizer.SetMinimumStepLength(0.001)
optimizer.SetNumberOfIterations(100)  # Diminuez ici pour tester l'effet

#Définir la métrique
metric = itk.MeanSquaresImageToImageMetricv4[FixedImageType, MovingImageType].New()

#Définir l'interpolateur
interpolator = itk.LinearInterpolateImageFunction[FixedImageType, itk.D].New()
metric.SetFixedInterpolator(interpolator)

#Préparer la méthode de recalage
registration = itk.ImageRegistrationMethodv4[FixedImageType, MovingImageType].New()
registration.SetFixedImage(fixed_image)
registration.SetMovingImage(moving_image)
registration.SetInitialTransform(initial_transform)
registration.SetMetric(metric)
registration.SetOptimizer(optimizer)

#Lancer
registration.Update()

#Résultats
final_transform = registration.GetTransform()
final_parameters = final_transform.GetParameters()
translation_x = final_parameters[0]
translation_y = final_parameters[1]

print("Résultat :")
print(f" Translation X = {translation_x:.2f}")
print(f" Translation Y = {translation_y:.2f}")
print(f" Iterations    = {optimizer.GetCurrentIteration()}")
print(f" Metric value  = {optimizer.GetValue():.6f}")

#Recalage
resampler = itk.ResampleImageFilter[MovingImageType, FixedImageType].New()
resampler.SetInput(moving_image)
resampler.SetTransform(final_transform)
resampler.SetUseReferenceImage(True)
resampler.SetReferenceImage(fixed_image)
resampler.SetDefaultPixelValue(0)
resampler.Update()

#Sauvegarde
aligned_image = resampler.GetOutput()

itk.imwrite(aligned_image, "aligned_image.nii.gz")

Résultat :
 Translation X = -0.83
 Translation Y = -3.54
 Iterations    = 20
 Metric value  = 11180.776679


In [208]:
ImageTypeOut = itk.Image[itk.UC, 3]
gradient = itk.GradientAnisotropicDiffusionImageFilter.New(fixed_image)
gradient.SetNumberOfIterations(5)
gradient.SetTimeStep(0.05)
gradient.SetConductanceParameter(3.0)
gradient.Update()
smooth_image = gradient.GetOutput()

confidence_filter = itk.ConfidenceConnectedImageFilter.New(smooth_image)
confidence_filter.SetInitialNeighborhoodRadius(2)
confidence_filter.SetMultiplier(2.3)
confidence_filter.SetNumberOfIterations(2)
confidence_filter.SetReplaceValue(1)

seed = itk.Index[3]()
seed[0], seed[1], seed[2] = 90, 60, 70
confidence_filter.AddSeed(seed)

confidence_filter.Update()
segmentation = confidence_filter.GetOutput()

final = itk.RescaleIntensityImageFilter[type(segmentation), ImageTypeOut].New()
final.SetInput(segmentation)
final.SetOutputMinimum(0)
final.SetOutputMaximum(255)
final.Update()
final_image = final.GetOutput()

itk.imwrite(final_image, "segmentation_fixed.nii.gz")

In [206]:
ImageTypeOut = itk.Image[itk.UC, 3]
gradient = itk.GradientAnisotropicDiffusionImageFilter.New(aligned_image)
gradient.SetNumberOfIterations(5)
gradient.SetTimeStep(0.05)
gradient.SetConductanceParameter(3.0)
gradient.Update()
smooth_image = gradient.GetOutput()

confidence_filter = itk.ConfidenceConnectedImageFilter.New(smooth_image)
confidence_filter.SetInitialNeighborhoodRadius(2) 
confidence_filter.SetMultiplier(2.5)             
confidence_filter.SetNumberOfIterations(2)        
confidence_filter.SetReplaceValue(1)

seed = itk.Index[3]()
seed[0], seed[1], seed[2] = 90, 60, 70
confidence_filter.AddSeed(seed)

confidence_filter.Update()
segmentation_image = confidence_filter.GetOutput()

final = itk.RescaleIntensityImageFilter[type(segmentation_image), ImageTypeOut].New()
final.SetInput(segmentation_image)
final.SetOutputMinimum(0)
final.SetOutputMaximum(255)
final.Update()
final_image = final.GetOutput()

itk.imwrite(final_image, "segmentation_aligned.nii.gz")